In [1]:
import ROOT as r
from ROOT import TFile, TTree, TH1F, TCanvas, TMath, TLegend
from array import array
import matplotlib as mpl
import matplotlib.pyplot as plt
import uproot
import sys, os
import pandas as pd
import numpy as np
#plt.style.use('default')

%matplotlib inline
%matplotlib widget
%jsroot on 

Welcome to JupyROOT 6.24/00


In [2]:
samples = ['Lb_Lcmunu','Lb_Lctaunu','Lb_LcDs','Lb_Lc2593munu','Lb_Lc2593taunu','Lb_Lc2593Ds',
             'Lb_Lc2625munu','Lb_Lc2625taunu','Lb_Lc2625Ds']
polarities = ['MagUp','MagDown']
#folder = '/disk/lhcb_data2/RLcMuonic2016/MC_full_trueTrigger/'
#folderTO = '/disk/lhcb_data2/RLcMuonic2016/MC_TrackerOnly/'
folder = '/Users/annarita/cernbox/LHCbDatasets/MC_full_trueTrigger/'
folderTO = '/Users/annarita/cernbox/LHCbDatasets/MC_TrackerOnly/'
muMotherID = {'Lb_Lcmunu':[5122],'Lb_Lc2625munu':[5122],'Lb_Lc2593munu':[5122],
             'Lb_Lctaunu':[13],'Lb_Lc2625taunu':[13],'Lb_Lc2593taunu':[13],
             'Lb_LcDs':[431,421,411],'Lb_Lc2625Ds':[431,421,411],'Lb_Lc2593Ds':[431,421,411]}
variables = ['Lb_IPCHI2_OWNPV', 'Lb_IP_OWNPV', 'Lb_TRUEP_X','Lb_TRUEP_Y','Lb_TRUEP_Z','Lb_BKGCAT','Lc_BKGCAT',
            'Lb_TRUEID', 'Lc_TRUEID', 'mu_TRUEID', 'mu_MC_MOTHER_ID','Lb_P','Lb_PT','Lc_M','mu_PT','mu_P','p_P',
             'p_PT','K_P','K_PT','pi_P','pi_PT','FitVar_El_mLc','FitVar_q2_mLc','FitVar_Mmiss2_mLc']

In [3]:
def TruthMatchMotherID(sample,branches):
    if sample not in ['Lb_LcDs','Lb_Lc2625Ds','Lb_Lc2593Ds']:
        MuonMother_mask = abs(branches['mu_MC_MOTHER_ID'])==muMotherID[sample][0]
    else:
        MuonMother_mask = (abs(branches['mu_MC_MOTHER_ID'])==muMotherID[sample][0])|(abs(branches['mu_MC_MOTHER_ID'])==muMotherID[sample][1])|(abs(branches['mu_MC_MOTHER_ID'])==muMotherID[sample][2])
    return MuonMother_mask

In [4]:
def TruthMatch(sample, branches):
    TruthMatch_mask = (branches['Lc_BKGCAT']<30) & (branches['Lb_BKGCAT']<50) & (abs(branches['Lb_TRUEID'])==5122) & (abs(branches['Lc_TRUEID'])==4122)
    return TruthMatch_mask

In [5]:
#samples = ['Lb_Lcmunu']
branches, branchesTO = {}, {}
MuonMother_mask, MuonMother_mask_TO = {}, {}
TruthMatch_mask, TruthMatch_mask_TO = {}, {}
for sample in samples:
    f = uproot.open(folder+sample+'_MagUp_full.root')
    t = f['tupleout/DecayTree']
    fTO = uproot.open(folderTO+sample+'_MagUp.root')
    tTO = fTO['tupleout/DecayTree']
    branches[sample] = t.arrays(variables, library="np")
    branchesTO[sample] = tTO.arrays(variables, library="np")
    MuonMother_mask[sample] = TruthMatchMotherID(sample,branches[sample])
    MuonMother_mask_TO[sample] = TruthMatchMotherID(sample,branchesTO[sample])
    TruthMatch_mask[sample] = TruthMatch(sample, branches[sample])
    TruthMatch_mask_TO[sample] = TruthMatch(sample, branchesTO[sample])

In [6]:
fig = {}
plotvars = {'Lb_IPCHI2_OWNPV':[0,5e3], 'Lb_IP_OWNPV':[0,25], 'Lb_TRUEP_X':[-100e3,100e3],'Lb_TRUEP_Y':[-100e3,100e3],
            'Lb_TRUEP_Z':[0,5e5],'Lb_P':[0,5e5],'Lb_PT':[0,80e3],'Lc_M':[2200,2380],'mu_PT':[0,5e4],'mu_P':[0,8e5],
            'p_P':[0,6e5],'p_PT':[0,3e4],'K_P':[0,5e5],'K_PT':[0,3e4],'pi_P':[0,5e5],'pi_PT':[0,3e4],
            'FitVar_El_mLc':[0,2e4],'FitVar_q2_mLc':[-2e6,12e6],'FitVar_Mmiss2_mLc':[-2e6,12e6]}

In [7]:
figure, axis = {}, {}
position = [[0,0],[0,1],[0,2],[1,0],[1,1],[1,2],[3,1],[3,2],[3,3]]
for plotvar in plotvars.keys():
    figure[plotvar],axis[plotvar] = plt.subplots(3, 3,figsize=(15,15))
    for i,sample in enumerate(samples):
        print(sample,i)
        #print(position[i])
        #print(position[i][0],position[i][1])
        axis[plotvar][position[i][0],position[i][1]].hist(branches[sample][plotvar][TruthMatch_mask[sample] & MuonMother_mask[sample]].flatten(),bins=50,range=(plotvars[plotvar][0], plotvars[plotvar][1]),density=True,alpha=0.5,label='Full')
        axis[plotvar][position[i][0],position[i][1]].hist(branchesTO[sample][plotvar][TruthMatch_mask_TO[sample] & MuonMother_mask_TO[sample]].flatten(),bins=50,range=(plotvars[plotvar][0], plotvars[plotvar][1]),density=True,alpha=0.5,label='TO')
        axis[plotvar][position[i][0],position[i][1]].set_title(plotvar+'_'+sample)
        axis[plotvar][position[i][0],position[i][1]].legend(loc='upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Lb_Lcmunu 0
Lb_Lctaunu 1
Lb_LcDs 2


/opt/anaconda3/envs/lhcbana/lib/python3.7/site-packages/numpy/lib/histograms.py:905: RuntimeWarning: invalid value encountered in true_divide
  return n/db/n.sum(), bin_edges


Lb_Lc2593munu 3
Lb_Lc2593taunu 4
Lb_Lc2593Ds 5
Lb_Lc2625munu 6


IndexError: index 3 is out of bounds for axis 0 with size 3

In [12]:
for sample in samples:
    fig[sample] = {}
    for plotvar in plotvars.keys():
        fig[sample][plotvar] = plt.figure(figsize=(8,6))
        plt.hist(branches[sample][plotvar][TruthMatch_mask[sample] & MuonMother_mask[sample]].flatten(),bins=50, range=(plotvars[plotvar][0], plotvars[plotvar][1]),density=True,alpha=0.5,label='Full')
        plt.hist(branchesTO[sample][plotvar][TruthMatch_mask_TO[sample] & MuonMother_mask_TO[sample]].flatten(),bins=50, range=(plotvars[plotvar][0], plotvars[plotvar][1]), density=True,alpha=0.5,label='TO')
        plt.legend(loc='upper right')
        plt.yscale('log')
        fig[sample][plotvar].show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

/opt/anaconda3/envs/lhcbana/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  after removing the cwd from sys.path.


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['Lb_IPCHI2_OWNPV'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0, 5000),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['Lb_IPCHI2_OWNPV'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0, 5000), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['Lb_P'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['Lb_P'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['FitVar_El_mLc'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0, 5000),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['FitVar_El_mLc'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0, 5000), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    #plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['FitVar_q2_mLc'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0,12e6),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['FitVar_q2_mLc'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0,12e6), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    #plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['FitVar_Mmiss2_mLc'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0,12e6),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['FitVar_Mmiss2_mLc'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0,12e6), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['Lb_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0,1e3),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['Lb_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0,1e3), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['Lb_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0,5000),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['Lb_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0,5e3), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    plt.figure(figsize=(8,6))
    plt.hist(branches['mu_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50, range=(0,8e4),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO['mu_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50, range=(0,8e4), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()

In [ ]:
    # Initialise the subplot function using number of rows and columns
    figure, axis = plt.subplots(3, 2,figsize=(15,15))

    # For Sine Function
    axis[0, 0].hist(branches['p_P'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[0, 0].hist(branchesTO['p_P'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[0, 0].set_title("p_P")
    axis[0, 0].legend(loc='upper right')
    
    # For Cosine Function
    axis[0, 1].hist(branches['p_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[0, 1].hist(branchesTO['p_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[0, 1].set_title("p_PT")

    # For Tangent Function
    axis[1, 0].hist(branches['K_P'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[1, 0].hist(branchesTO['K_P'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[1, 0].set_title("K_P")
    
    axis[1, 1].hist(branches['K_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[1, 1].hist(branchesTO['K_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[1, 1].set_title("K_PT")

    # For Tanh Function
    axis[2, 0].hist(branches['pi_P'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[2, 0].hist(branchesTO['pi_P'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[2, 0].set_title("pi_P")
    
    axis[2, 1].hist(branches['pi_PT'][TruthMatch_mask & MuonMother_mask].flatten(),bins=50,density=True,alpha=0.5,label='Full')
    axis[2, 1].hist(branchesTO['pi_PT'][TruthMatch_mask_TO & MuonMother_mask_TO].flatten(),bins=50,density=True,alpha=0.5,label='TO')
    axis[2, 1].set_title("pi_PT")

    # Combine all the operations and display
    plt.show()

In [ ]:
samples = ['Lb_Lcmunu']
branches, branchesTO = {}, {}
MuonMother_mask, MuonMother_mask_TO = {}, {}
TruthMatch_mask, TruthMatch_mask_TO = {}, {}
for sample in samples:
    f = uproot.open(folder+sample+'_MagUp_full.root')
    t = f['tupleout/DecayTree']
    fTO = uproot.open(folderTO+sample+'_MagUp.root')
    tTO = fTO['tupleout/DecayTree']
    branches[sample] = t.arrays(variables, library="np")
    branchesTO[sample] = tTO.arrays(variables, library="np")
    MuonMother_mask[sample] = TruthMatchMotherID(sample,branches[sample])
    MuonMother_mask_TO[sample] = TruthMatchMotherID(sample,branchesTO[sample])
    TruthMatch_mask[sample] = TruthMatch(sample, branches[sample])
    TruthMatch_mask_TO[sample] = TruthMatch(sample, branchesTO[sample])

In [ ]:
branches['Lb_Lcmunu']

In [ ]:
MuonMother_mask

In [ ]:
for sample in samples:
    plt.figure(figsize=(8,6))
    plt.hist(branches[sample]['Lb_IPCHI2_OWNPV'][TruthMatch_mask[sample] & MuonMother_mask[sample]].flatten(),bins=50, range=(0, 5000),density=True,alpha=0.5,label='Full')
    plt.hist(branchesTO[sample]['Lb_IPCHI2_OWNPV'][TruthMatch_mask_TO[sample] & MuonMother_mask_TO[sample]].flatten(),bins=50, range=(0, 5000), density=True,alpha=0.5,label='TO')
    plt.legend(loc='upper right')
    plt.yscale('log')
    plt.show()